# INTRO

## Import Basic Libraries

In [1]:
import pandas as pd
import numpy as np


## Load Data

In [2]:

data = pd.read_csv('../data/raw/Loan_Default.csv')
data.head(5)

,ID,year,loan_limit,Gender,approv_in_adv,loan_type,loan_purpose,Credit_Worthiness,open_credit,business_or_commercial,...,credit_type,Credit_Score,co-applicant_credit_type,age,submission_of_application,LTV,Region,Security_Type,Status,dtir1
0,24890,2019,cf,Sex Not Available,nopre,type1,p1,l1,nopc,nob/c,...,EXP,758,CIB,25-34,to_inst,98.728814,south,direct,1,45.0
1,24891,2019,cf,Male,nopre,type2,p1,l1,nopc,b/c,...,EQUI,552,EXP,55-64,to_inst,NaN,North,direct,1,NaN
2,24892,2019,cf,Male,pre,type1,p1,l1,nopc,nob/c,...,EXP,834,CIB,35-44,to_inst,80.019685,south,direct,0,46.0
3,24893,2019,cf,Male,nopre,type1,p4,l1,nopc,nob/c,...,EXP,587,CIB,45-54,not_inst,69.376900,North,direct,0,42.0
4,24894,2019,cf,Joint,pre,type1,p1,l1,nopc,nob/c,...,CRIF,602,EXP,25-34,not_inst,91.886544,North,direct,0,39.0


## Data Exploration

In [3]:
data.describe()

,ID,year,loan_amount,rate_of_interest,Interest_rate_spread,Upfront_charges,term,property_value,income,Credit_Score,LTV,Status,dtir1
count,148670.000000,148670.0,1.486700e+05,112231.000000,112031.000000,109028.000000,148629.000000,1.335720e+05,139520.000000,148670.000000,133572.000000,148670.000000,124549.000000
mean,99224.500000,2019.0,3.311177e+05,4.045476,0.441656,3224.996127,335.136582,4.978935e+05,6957.338876,699.789103,72.746457,0.246445,37.732932
std,42917.476598,0.0,1.839093e+05,0.561391,0.513043,3251.121510,58.409084,3.599353e+05,6496.586382,115.875857,39.967603,0.430942,10.545435
min,24890.000000,2019.0,1.650000e+04,0.000000,-3.638000,0.000000,96.000000,8.000000e+03,0.000000,500.000000,0.967478,0.000000,5.000000
25%,62057.250000,2019.0,1.965000e+05,3.625000,0.076000,581.490000,360.000000,2.680000e+05,3720.000000,599.000000,60.474860,0.000000,31.000000
50%,99224.500000,2019.0,2.965000e+05,3.990000,0.390400,2596.450000,360.000000,4.180000e+05,5760.000000,699.000000,75.135870,0.000000,39.000000
75%,136391.750000,2019.0,4.365000e+05,4.375000,0.775400,4812.500000,360.000000,6.280000e+05,8520.000000,800.000000,86.184211,0.000000,45.000000
max,173559.000000,2019.0,3.576500e+06,8.000000,3.357000,60000.000000,360.000000,1.650800e+07,578580.000000,900.000000,7831.250000,1.000000,61.000000


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148670 entries, 0 to 148669
Data columns (total 34 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   ID                         148670 non-null  int64  
 1   year                       148670 non-null  int64  
 2   loan_limit                 145326 non-null  object 
 3   Gender                     148670 non-null  object 
 4   approv_in_adv              147762 non-null  object 
 5   loan_type                  148670 non-null  object 
 6   loan_purpose               148536 non-null  object 
 7   Credit_Worthiness          148670 non-null  object 
 8   open_credit                148670 non-null  object 
 9   business_or_commercial     148670 non-null  object 
 10  loan_amount                148670 non-null  int64  
 11  rate_of_interest           112231 non-null  float64
 12  Interest_rate_spread       112031 non-null  float64
 13  Upfront_charges            10

In [5]:
data.sample(10)

,ID,year,loan_limit,Gender,approv_in_adv,loan_type,loan_purpose,Credit_Worthiness,open_credit,business_or_commercial,...,credit_type,Credit_Score,co-applicant_credit_type,age,submission_of_application,LTV,Region,Security_Type,Status,dtir1
62130,87020,2019,cf,Male,pre,type1,p4,l1,nopc,nob/c,...,CRIF,564,CIB,35-44,not_inst,69.625604,North,direct,0,23.0
51678,76568,2019,cf,Sex Not Available,nopre,type1,p4,l1,nopc,nob/c,...,CRIF,768,EXP,55-64,to_inst,70.712209,south,direct,0,40.0
71259,96149,2019,cf,Male,nopre,type1,p4,l2,nopc,nob/c,...,CRIF,502,CIB,35-44,to_inst,80.021834,North,direct,0,31.0
102057,126947,2019,cf,Joint,pre,type1,NaN,l1,nopc,nob/c,...,CIB,548,EXP,25-34,not_inst,94.597990,North,direct,0,29.0
30629,55519,2019,cf,Male,nopre,type1,p1,l1,nopc,nob/c,...,CIB,612,CIB,<25,not_inst,79.362416,North,direct,0,47.0
57575,82465,2019,cf,Joint,nopre,type2,p3,l1,nopc,b/c,...,EXP,502,EXP,>74,to_inst,44.140625,south,direct,0,37.0
68166,93056,2019,cf,Sex Not Available,nopre,type1,p1,l1,nopc,nob/c,...,CIB,604,CIB,45-54,to_inst,95.542636,south,direct,0,44.0
82512,107402,2019,cf,Female,nopre,type1,p3,l1,nopc,nob/c,...,CIB,809,CIB,55-64,to_inst,84.090909,North,direct,1,48.0
147352,172242,2019,cf,Male,nopre,type1,p1,l1,nopc,nob/c,...,CIB,631,CIB,65-74,to_inst,86.764706,North,direct,0,44.0
54225,79115,2019,NaN,Joint,nopre,type1,p1,l1,nopc,nob/c,...,CIB,556,EXP,45-54,not_inst,78.979239,North,direct,0,26.0


In [6]:
print(data.shape)
print(data.columns)

(148670, 34)
Index(['ID', 'year', 'loan_limit', 'Gender', 'approv_in_adv', 'loan_type',
       'loan_purpose', 'Credit_Worthiness', 'open_credit',
       'business_or_commercial', 'loan_amount', 'rate_of_interest',
       'Interest_rate_spread', 'Upfront_charges', 'term', 'Neg_ammortization',
       'interest_only', 'lump_sum_payment', 'property_value',
       'construction_type', 'occupancy_type', 'Secured_by', 'total_units',
       'income', 'credit_type', 'Credit_Score', 'co-applicant_credit_type',
       'age', 'submission_of_application', 'LTV', 'Region', 'Security_Type',
       'Status', 'dtir1'],
      dtype='object')


In [7]:
data.isnull().sum()

ID                               0
year                             0
loan_limit                    3344
Gender                           0
approv_in_adv                  908
loan_type                        0
loan_purpose                   134
Credit_Worthiness                0
open_credit                      0
business_or_commercial           0
loan_amount                      0
rate_of_interest             36439
Interest_rate_spread         36639
Upfront_charges              39642
term                            41
Neg_ammortization              121
interest_only                    0
lump_sum_payment                 0
property_value               15098
construction_type                0
occupancy_type                   0
Secured_by                       0
total_units                      0
income                        9150
credit_type                      0
Credit_Score                     0
co-applicant_credit_type         0
age                            200
submission_of_applic

# EDA

## Data Cleaning: Dropping and Handling Missing Values

### What We Are Doing
1. **Dropping Unnecessary Columns**  
   - Removing columns that are not needed for analysis:  
     `'loan_limit'`, `'rate_of_interest'`, `'Interest_rate_spread'`, `'Upfront_charges'`, `'property_value'`, `'dtir1'`, and `'LTV'`.  
   - This reduces dimensionality and improves efficiency.

2. **Handling Missing Values**  
   - Iterating through all columns in the dataset:
     - If a column has missing values:
       - If it's numerical (`float64` or `int64`), we fill missing values with the **median** to avoid skewing the data.
       - If it's categorical, we fill missing values with the **most frequent value** (mode).
   - This ensures that no missing values remain, preventing errors in analysis or modeling.

3. **Validation**  
   - Printing the count of missing values after imputation to confirm that all missing data has been handled.


In [8]:
# Dropping unnecessary columns
unnecessary_columns = ['loan_limit', 'rate_of_interest', 'Interest_rate_spread', 
                        'Upfront_charges', 'property_value', 'dtir1', 'LTV']
data.drop(unnecessary_columns, axis=1, inplace=True)
data.head()

,ID,year,Gender,approv_in_adv,loan_type,loan_purpose,Credit_Worthiness,open_credit,business_or_commercial,loan_amount,...,total_units,income,credit_type,Credit_Score,co-applicant_credit_type,age,submission_of_application,Region,Security_Type,Status
0,24890,2019,Sex Not Available,nopre,type1,p1,l1,nopc,nob/c,116500,...,1U,1740.0,EXP,758,CIB,25-34,to_inst,south,direct,1
1,24891,2019,Male,nopre,type2,p1,l1,nopc,b/c,206500,...,1U,4980.0,EQUI,552,EXP,55-64,to_inst,North,direct,1
2,24892,2019,Male,pre,type1,p1,l1,nopc,nob/c,406500,...,1U,9480.0,EXP,834,CIB,35-44,to_inst,south,direct,0
3,24893,2019,Male,nopre,type1,p4,l1,nopc,nob/c,456500,...,1U,11880.0,EXP,587,CIB,45-54,not_inst,North,direct,0
4,24894,2019,Joint,pre,type1,p1,l1,nopc,nob/c,696500,...,1U,10440.0,CRIF,602,EXP,25-34,not_inst,North,direct,0


In [9]:
# Handling missing values
for column in data.columns:
    if data[column].isnull().sum() > 0:
        if data[column].dtype in ['float64', 'int64']:  
            data[column] = data[column].fillna(data[column].median())  # Explicit assignment
        else:  
            data[column] = data[column].fillna(data[column].mode()[0])  # Explicit assignment

print(data.isnull().sum())

ID                           0
year                         0
Gender                       0
approv_in_adv                0
loan_type                    0
loan_purpose                 0
Credit_Worthiness            0
open_credit                  0
business_or_commercial       0
loan_amount                  0
term                         0
Neg_ammortization            0
interest_only                0
lump_sum_payment             0
construction_type            0
occupancy_type               0
Secured_by                   0
total_units                  0
income                       0
credit_type                  0
Credit_Score                 0
co-applicant_credit_type     0
age                          0
submission_of_application    0
Region                       0
Security_Type                0
Status                       0
dtype: int64


In [10]:
data['Region'].value_counts()

Region
North         74722
south         64016
central        8697
North-East     1235
Name: count, dtype: int64

## Encoding:


In [11]:
data = pd.get_dummies(data, columns=['Region'], drop_first=True)
data['Region'].value_counts()


KeyError: 'Region'